# MNIST 다중 클래스 분류 (PyTorch 활용)

## 1. 들어가며

NumPy로 신경망의 기본 원리를 깊이 이해했고, Keras를 사용하여 프레임워크의 편리함을 경험했습니다. 이제 PyTorch를 사용하여 MNIST 손글씨 숫자 데이터를 분류하는 다중 클래스 분류 문제를 해결해 보겠습니다.

PyTorch는 유연성이 뛰어나 연구 개발에 많이 사용되며, "Pythonic"한 코딩 스타일을 선호하는 분들에게 인기가 많습니다. Keras와는 달리 학습 루프를 직접 작성해야 하므로, 신경망 내부 동작 과정을 더 명확하게 이해하는 데 도움이 될 수 있습니다.

이번 노트북에서는 PyTorch를 사용하여 다음 내용들을 학습합니다.

1.  **PyTorch를 이용한 MNIST 데이터셋 로딩 및 전처리**
2.  **데이터 로더(DataLoader)를 사용한 미니 배치 처리**
3.  **다중 클래스 분류를 위한 신경망 모델 정의 (`torch.nn.Module`)**
4.  **다중 클래스 분류를 위한 손실 함수 (`nn.CrossEntropyLoss`)**
5.  **옵티마이저 (`torch.optim.Adam`) 사용**
6.  **PyTorch에서의 학습 루프 (순전파, 손실 계산, 역전파, 옵티마이저 스텝)**
7.  **모델 평가 및 예측 (`model.eval()`, `torch.no_grad()`)**
8.  **과적합 방지를 위한 Dropout 적용**


In [ ]:
# 필요한 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader # 데이터셋 및 로더를 위해 필요
import torchvision # 이미지 관련 데이터셋 및 변환을 위해 필요
import torchvision.transforms as transforms # 이미지 변환을 위해 필요
import numpy as np
import matplotlib.pyplot as plt # 학습 과정 시각화를 위해 필요

# GPU 사용 가능 여부 확인 및 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


*   `torchvision`: PyTorch에서 이미지 데이터셋 (MNIST, CIFAR10 등)과 이미지 변환(Transform)을 편리하게 사용할 수 있도록 제공하는 라이브러리입니다.
*   `torch.utils.data.Dataset`, `DataLoader`: PyTorch에서 데이터를 효율적으로 불러오고 미니 배치 단위로 제공하는 핵심 클래스입니다.
*   `device`: 모델과 데이터를 CPU 또는 GPU로 이동시키기 위해 설정합니다. GPU가 있다면 학습 속도를 크게 높일 수 있습니다.

## 2. MNIST 데이터셋 로딩 및 준비 (PyTorch 스타일)

PyTorch에서는 `torchvision.datasets`를 사용하여 MNIST 데이터를 쉽게 로드하고, `torchvision.transforms`를 사용하여 필요한 전처리를 적용합니다. `DataLoader`는 이 데이터셋을 기반으로 미니 배치 단위의 데이터를 제공합니다.

### 2.1 데이터 로딩 및 변환 (Transform)

`transforms.ToTensor()`는 PIL Image나 NumPy 배열 데이터를 PyTorch Tensor로 변환하고, 픽셀 값을 자동으로 [0, 1] 범위로 스케일링(정규화) 해줍니다. 이는 Keras에서 `astype('float32') / 255.0` 했던 것과 동일한 효과입니다.

`transforms.Normalize()`는 픽셀 값을 평균 0, 표준 편차 1을 갖도록 추가적인 정규화를 수행합니다. 이는 모델 학습 안정성에 도움이 됩니다. MNIST 데이터셋의 평균과 표준 편차는 알려져 있습니다.


In [ ]:
# 데이터셋에 적용할 변환 정의
# ToTensor(): 이미지를 Tensor로 변환하고 [0, 1]로 정규화
# Normalize(): 평균 0.1307, 표준 편차 0.3081로 정규화
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # MNIST 데이터셋의 평균과 표준편차
])

# MNIST 학습 및 테스트 데이터셋 로딩
# root: 데이터셋을 저장할 경로
# train=True: 학습 데이터셋, train=False: 테스트 데이터셋
# download=True: 데이터셋이 없으면 다운로드
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

print(f"학습 데이터셋 크기: {len(train_dataset)}")
print(f"테스트 데이터셋 크기: {len(test_dataset)}")


### 2.2 데이터 로더 (DataLoader) 설정 (미니 배치)

`DataLoader`는 데이터셋에서 미니 배치 크기만큼 데이터를 자동으로 가져와서 Tensor 형태로 제공합니다. 학습 시에는 `shuffle=True`로 설정하여 데이터를 무작위로 섞어 모델이 데이터 순서에 의존하지 않도록 합니다.


In [ ]:
# 데이터 로더 설정
batch_size = 64 # 한 번에 처리할 미니 배치 크기

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False) # 테스트 시에는 섞을 필요 없음

# 데이터 로더에서 첫 번째 배치 데이터 형태 확인
# iter(train_loader).next() 대신 next(iter(train_loader)) 사용 (Python 3.6 이상 권장)
images, labels = next(iter(train_loader))

print(f"첫 번째 배치 이미지 형태: {images.shape}") # (batch_size, 채널, 높이, 너비) -> (64, 1, 28, 28)
print(f"첫 번째 배치 레이블 형태: {labels.shape}") # (batch_size) -> (64)
print(f"첫 번째 배치 레이블 (처음 5개): {labels[:5]}") # 레이블은 원본 정수 형태입니다.


*   PyTorch의 이미지 Tensor는 `(Batch Size, Channels, Height, Width)` 순서입니다. MNIST는 흑백 이미지이므로 채널이 1입니다.
*   레이블은 다중 클래스 분류를 위해 정수 형태(0~9) 그대로 사용합니다. 이는 Keras의 `categorical_crossentropy`와 달리 PyTorch의 `nn.CrossEntropyLoss`는 원본 정수 레이블을 입력받기 때문입니다.

### 2.3 데이터 시각화 (DataLoader에서 가져온 샘플 확인)

`DataLoader`에서 가져온 배치의 첫 번째 이미지를 시각화해 봅시다. `Normalize`를 적용했기 때문에 픽셀 값이 음수가 될 수도 있지만, 시각화 라이브러리는 자동으로 스케일링하여 보여줍니다.


In [ ]:
# 첫 번째 배치에서 이미지 하나를 시각화 (정규화된 값일 수 있습니다)
# Tensor를 NumPy 배열로 변환하고 채널 차원 제거 (1, 28, 28) -> (28, 28)
img_tensor = images[0].squeeze() # 채널 차원이 1이면 제거
img_numpy = img_tensor.numpy()

plt.imshow(img_numpy, cmap='gray')
plt.title(f"Label: {labels[0].item()}") # Tensor의 값을 Python 숫자로 가져옴
plt.axis('off')
plt.show()


## 3. 신경망 모델 정의 (PyTorch `nn.Module`)

PyTorch에서 신경망 모델은 `torch.nn.Module` 클래스를 상속받아 정의합니다.

*   `__init__` 메서드에서는 모델의 구성 요소(레이어 등)를 정의합니다.
*   `forward` 메서드에서는 입력 데이터가 모델을 통과하는 순서를 정의합니다. Keras Sequential API의 레이어 순서와 같습니다.

여기서도 Keras 예제와 유사하게 입력층(이미지 플래트닝), 은닉층(ReLU), 출력층(Softmax는 손실 함수 내장)으로 구성합니다.


In [ ]:
# 모델 파라미터 설정
input_size = 28 * 28 # 784
hidden_size = 128    # 은닉층 뉴런 수 (변경 가능)
num_classes = 10     # 출력층 뉴런 수 (0~9 클래스)
learning_rate = 0.001 # 학습률 (Adam 사용 시 이 값부터 시작해 보는 경우가 많습니다)
epochs = 10          # 전체 데이터셋 반복 횟수

# 신경망 모델 클래스 정의
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # 첫 번째 선형 레이어 (입력 -> 은닉)
        self.relu = nn.ReLU()                       # ReLU 활성화 함수
        self.fc2 = nn.Linear(hidden_size, num_classes) # 두 번째 선형 레이어 (은닉 -> 출력)
        # Softmax는 CrossEntropyLoss 안에 포함되어 있으므로 마지막에 따로 적용하지 않습니다.

    def forward(self, x):
        # 이미지를 (batch_size, 1, 28, 28)에서 (batch_size, 784)로 펼침
        x = x.reshape(-1, input_size) # -1은 batch_size에 맞춰 자동으로 계산

        out = self.fc1(x)     # 첫 번째 선형 변환
        out = self.relu(out)  # ReLU 활성화
        out = self.fc2(out)     # 두 번째 선형 변환 (출력층)
        # 마지막 출력은 클래스별 점수 (logits) 입니다.
        return out

# 모델 인스턴스 생성 및 디바이스(CPU/GPU)로 이동
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

print("모델 구조:")
print(model)


*   `nn.Linear(in_features, out_features)`: 완전 연결(Fully Connected) 레이어입니다. `in_features`는 입력 뉴런 수, `out_features`는 출력 뉴런 수입니다. 가중치(W)와 편향(b)을 포함합니다.
*   `nn.ReLU()`: ReLU 활성화 함수입니다.
*   `reshape(-1, input_size)`: `forward` 메서드에서 이미지 Tensor의 형태를 학습에 사용할 수 있도록 펼쳐주는 부분입니다. `-1`은 배치 크기를 의미하며, PyTorch가 자동으로 계산합니다.

## 4. 손실 함수 및 옵티마이저 정의

PyTorch에서 손실 함수는 `torch.nn` 모듈에, 옵티마이저는 `torch.optim` 모듈에 있습니다.

*   **손실 함수:** 다중 클래스 분류에는 `nn.CrossEntropyLoss`를 사용합니다. 이 함수는 모델의 출력(Logits, 즉 Softmax 적용 전의 원시 점수)과 원본 정수 레이블을 입력받아 손실을 계산합니다. 내부적으로 Softmax와 Negative Log Likelihood Loss를 결합한 형태입니다.
*   **옵티마이저:** Adam 옵티마이저를 사용합니다. `optim.Adam(model.parameters(), lr=learning_rate)` 형태로 모델의 모든 학습 가능한 파라미터와 학습률을 전달하여 생성합니다.


In [ ]:
# 손실 함수 정의: Cross Entropy Loss
criterion = nn.CrossEntropyLoss()

# 옵티마이저 정의: Adam 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("손실 함수 및 옵티마이저 정의 완료!")


## 5. 모델 학습 (PyTorch 학습 루프)

PyTorch에서는 학습 루프를 직접 작성해야 합니다. 기본적인 학습 루프는 다음과 같습니다.

1.  **Epoch 반복:** 전체 데이터셋을 몇 번 학습할지 결정합니다.
2.  **Batch 반복:** 각 Epoch 내에서 `DataLoader`를 통해 미니 배치 단위로 데이터를 가져옵니다.
3.  **데이터 디바이스 이동:** 배치 데이터를 CPU 또는 GPU(`device` 변수)로 이동시킵니다.
4.  **기울기 초기화:** 이전 배치에서 계산된 기울기를 0으로 초기화합니다 (`optimizer.zero_grad()`).
5.  **순전파:** 모델에 입력 데이터를 넣어 예측 값을 계산합니다 (`outputs = model(inputs)`).
6.  **손실 계산:** 예측 값과 실제 레이블을 사용하여 손실 함수 값을 계산합니다 (`loss = criterion(outputs, labels)`).
7.  **역전파:** 손실 함수 값을 바탕으로 모델의 모든 학습 가능한 파라미터에 대한 기울기를 계산합니다 (`loss.backward()`).
8.  **가중치 업데이트:** 계산된 기울기를 사용하여 옵티마이저가 모델의 가중치를 업데이트합니다 (`optimizer.step()`).
9.  **손실 및 정확도 기록:** 학습 과정을 모니터링하기 위해 배치별 손실과 정확도를 기록합니다.


In [ ]:
# 학습 과정 기록을 위한 리스트
train_losses = []
train_accuracies = []

print(f"모델 학습 시작 (Epochs: {epochs}, Batch Size: {batch_size})...")

# 모델을 학습 모드로 설정 (Dropout 등이 활성화됨)
model.train()

for epoch in range(epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # 데이터 로더를 통해 미니 배치 데이터 가져오기
    for i, (images, labels) in enumerate(train_loader):
        # 데이터를 지정된 디바이스로 이동 (CPU 또는 GPU)
        images = images.to(device)
        labels = labels.to(device)

        # ----------- 핵심 학습 단계 -----------
        # 1. 이전 기울기 초기화
        optimizer.zero_grad()

        # 2. 순전파: 입력 데이터를 모델에 통과시켜 출력(예측 값) 계산
        outputs = model(images) # 출력은 클래스별 점수(logits)

        # 3. 손실 계산: 예측 값(outputs)과 실제 레이블(labels) 비교
        loss = criterion(outputs, labels)

        # 4. 역전파: 손실을 바탕으로 기울기 계산
        loss.backward()

        # 5. 가중치 업데이트: 계산된 기울기를 사용하여 모델 파라미터 업데이트
        optimizer.step()
        # -------------------------------------

        running_loss += loss.item() # 배치별 손실 누적 (.item()으로 Tensor 값을 Python 숫자로 변환)

        # 정확도 계산 (학습 과정 모니터링용)
        # outputs에서 가장 높은 점수를 갖는 클래스 인덱스 찾기
        # dim=1은 두 번째 차원 (클래스 차원)을 기준으로 argmax 계산
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0) # 현재 배치의 샘플 수 누적
        correct_predictions += (predicted == labels).sum().item() # 예측과 실제가 일치하는 경우 누적

        # (선택 사항) 배치별 진행 상황 출력
        # if (i + 1) % 100 == 0: # 100 배치마다 출력
        #     print(f'  Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], '
        #           f'Loss: {loss.item():.4f}, Acc: {(correct_predictions/total_samples):.4f}')


    # 에포크 종료 후 평균 손실 및 정확도 계산 및 기록
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_samples

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {epoch_loss:.4f}, Average Accuracy: {epoch_accuracy:.4f}')

print("모델 학습 완료!")


이 학습 루프는 PyTorch를 사용할 때 가장 기본이 되는 형태입니다. 각 줄의 의미와 순서를 잘 이해하는 것이 중요합니다.

### 학습 과정 시각화

기록해 둔 에포크별 손실과 정확도를 그래프로 그려봅시다.


In [ ]:
# 학습 과정 시각화
plt.figure(figsize=(12, 4))

# 손실(Loss) 그래프
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses)
plt.title('Training Loss per Epoch')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.grid(True)

# 정확도(Accuracy) 그래프
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_accuracies)
plt.title('Training Accuracy per Epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid(True)

plt.show()


## 6. 모델 평가

학습이 완료된 모델의 성능을 테스트 데이터셋으로 평가합니다. 평가 시에는 기울기 계산이 필요 없으므로 `torch.no_grad()` 컨텍스트 매니저를 사용하고, 모델을 평가 모드(`model.eval()`)로 설정합니다. 평가 모드는 Dropout과 같은 특정 레이어의 작동 방식을 변경합니다.


In [ ]:
print("모델 평가 시작...")
# 모델을 평가 모드로 설정 (Dropout 등이 비활성화됨)
model.eval()

# 기울기 계산을 비활성화 (메모리 사용량 감소, 연산 속도 향상)
with torch.no_grad():
    correct_predictions = 0
    total_samples = 0
    running_test_loss = 0.0

    # 테스트 데이터 로더를 통해 배치 데이터 가져오기
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # 순전파 (예측)
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs, labels)
        running_test_loss += loss.item()

        # 정확도 계산
        _, predicted = torch.max(outputs.data, 1) # 가장 높은 점수를 갖는 클래스 인덱스
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item() # 예측과 실제가 일치하는 경우

    # 테스트 데이터셋 전체에 대한 평균 손실 및 정확도 계산
    test_loss = running_test_loss / len(test_loader)
    test_accuracy = correct_predictions / total_samples

    print(f"테스트 손실(Loss): {test_loss:.4f}")
    print(f"테스트 정확도(Accuracy): {test_accuracy:.4f}")

print("모델 평가 완료!")


테스트 정확도는 모델이 학습되지 않은 새로운 데이터에 대해 얼마나 잘 작동하는지 보여줍니다.

## 7. 모델 예측

학습된 모델로 개별 샘플에 대한 예측을 수행해 봅시다. 예측 시에도 `model.eval()`과 `torch.no_grad()`를 사용합니다. 모델 출력은 Logits(점수)이므로, 이를 확률로 변환하기 위해 Softmax를 적용한 후, 가장 높은 확률을 갖는 클래스를 선택합니다.


In [ ]:
# 테스트 데이터셋에서 이미지 샘플 하나 가져오기
# batch_size가 1인 새로운 DataLoader를 사용하거나, 기존 DataLoader에서 하나씩 가져옵니다.
# 여기서는 test_dataset에서 직접 샘플을 가져와서 배치 형태로 만듭니다.
sample_index = 15 # 예측해 볼 샘플 인덱스 (예: 15번째 이미지)
sample_image, sample_label = test_dataset[sample_index]

# 모델에 입력하기 위해 (채널, 높이, 너비) -> (배치, 채널, 높이, 너비) 형태로 변환
# unsqueeze(0)은 맨 앞에 차원을 하나 추가하여 배치 차원을 만듭니다.
sample_image_input = sample_image.unsqueeze(0).to(device)

# 모델을 평가 모드로 설정
model.eval()

# 기울기 계산 비활성화
with torch.no_grad():
    # 예측 수행
    output = model(sample_image_input) # 출력은 (1, 10) 형태의 logits

    # 예측 결과(logits)를 확률로 변환 (Softmax 적용)
    probabilities = torch.softmax(output, dim=1) # dim=1은 클래스 차원

    # 가장 높은 확률을 갖는 클래스 인덱스 찾기
    _, predicted_class = torch.max(probabilities, 1)

# 결과 출력
print(f"모델 예측 결과 (Logits): {output}")
print(f"모델 예측 결과 (Probabilities): {probabilities}")
print(f"예측된 클래스: {predicted_class.item()}") # Tensor 값을 Python 숫자로
print(f"실제 레이블: {sample_label}")

# 이미지 시각화
# Tensor를 NumPy 배열로 변환하고 채널 차원 제거
img_tensor = sample_image.squeeze()
img_numpy = img_tensor.numpy()

plt.imshow(img_numpy, cmap='gray')
plt.title(f"Predicted: {predicted_class.item()}, Actual: {sample_label}")
plt.axis('off')
plt.show()


## 8. 과적합 방지를 위한 Dropout 적용 (PyTorch)

Keras와 마찬가지로 PyTorch에서도 `nn.Dropout` 레이어를 사용하여 Dropout을 적용할 수 있습니다. `nn.Dropout(p=rate)` 형태로 사용하며, `p`는 드롭아웃 비율(0~1)입니다.

Dropout 레이어는 `__init__` 메서드에서 정의하고, `forward` 메서드에서 활성화 함수 뒤에 적용하는 것이 일반적입니다. PyTorch의 `nn.Dropout`은 `model.train()`일 때만 작동하고 `model.eval()`일 때는 자동으로 비활성화됩니다.

Dropout을 추가한 새로운 모델을 정의하고 학습시켜 봅시다.


In [ ]:
print("\n--- Dropout을 추가한 PyTorch 모델 구성 ---")

dropout_rate = 0.2 # 20%의 뉴런을 비활성화

# 신경망 모델 클래스 정의 (Dropout 추가)
class NeuralNetWithDropout(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout_rate=0.5):
        super(NeuralNetWithDropout, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_rate) # Dropout 레이어 추가
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # 이미지를 펼침
        x = x.reshape(-1, input_size)

        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out) # ReLU 활성화 후 Dropout 적용
        out = self.fc2(out)
        return out

# Dropout 포함 모델 인스턴스 생성 및 디바이스로 이동
model_with_dropout = NeuralNetWithDropout(input_size, hidden_size, num_classes, dropout_rate).to(device)

print("모델 구조 (Dropout 포함):")
print(model_with_dropout)

# Dropout 포함 모델에 대한 손실 함수 및 옵티마이저 정의
criterion_dropout = nn.CrossEntropyLoss()
optimizer_dropout = optim.Adam(model_with_dropout.parameters(), lr=learning_rate)

print("\nDropout 포함 모델 학습 시작...")

# 학습 과정 기록을 위한 리스트
train_losses_dropout = []
train_accuracies_dropout = []

# 모델을 학습 모드로 설정
model_with_dropout.train()

for epoch in range(epochs): # Keras 예제와 동일하게 10 에포크
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer_dropout.zero_grad()
        outputs = model_with_dropout(images)
        loss = criterion_dropout(outputs, labels)
        loss.backward()
        optimizer_dropout.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_samples

    train_losses_dropout.append(epoch_loss)
    train_accuracies_dropout.append(epoch_accuracy)

    print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {epoch_loss:.4f}, Average Accuracy: {epoch_accuracy:.4f}')

print("Dropout 포함 모델 학습 완료!")

# Dropout 포함 모델 평가
print("\nDropout 포함 모델 평가 시작...")
model_with_dropout.eval() # 평가 모드 설정
with torch.no_grad():
    correct_predictions = 0
    total_samples = 0
    running_test_loss = 0.0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_with_dropout(images)
        loss = criterion_dropout(outputs, labels)
        running_test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    test_loss_dropout = running_test_loss / len(test_loader)
    test_accuracy_dropout = correct_predictions / total_samples

    print(f"Dropout 포함 모델 테스트 손실(Loss): {test_loss_dropout:.4f}")
    print(f"Dropout 포함 모델 테스트 정확도(Accuracy): {test_accuracy_dropout:.4f}")

# 학습 과정 시각화 (Dropout 포함 모델)
plt.figure(figsize=(12, 4))

# 손실(Loss) 그래프
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses_dropout)
plt.title('Training Loss per Epoch (with Dropout)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.grid(True)

# 정확도(Accuracy) 그래프
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_accuracies_dropout)
plt.title('Training Accuracy per Epoch (with Dropout)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid(True)

plt.show()


MNIST 데이터셋은 상대적으로 단순하기 때문에 Dropout의 효과가 드라마틱하지 않을 수 있습니다. 하지만 더 크고 복잡한 데이터셋이나 모델에서는 과적합을 방지하고 테스트 성능을 향상시키는 데 중요한 역할을 합니다.

## 9. 마치며

PyTorch를 사용하여 MNIST 다중 클래스 분류 모델을 성공적으로 구현하고 학습시켰습니다! Keras와는 다른 PyTorch만의 데이터 처리 (`Dataset`, `DataLoader`, `transforms`) 및 모델 정의 (`nn.Module`, `nn.Linear`, `nn.ReLU`) 방식, 그리고 학습 루프를 직접 작성하는 경험을 해보았습니다. 또한, 다중 클래스 분류를 위한 `nn.CrossEntropyLoss`와 일반적인 옵티마이저인 `optim.Adam`, 그리고 과적합 방지를 위한 `nn.Dropout` 사용법을 익혔습니다.

이제 다음 단계로 나아가기 위한 탄탄한 기초를 PyTorch로도 다지게 되었습니다. 앞으로 PyTorch를 사용하여 다양한 딥러닝 모델(CNN, RNN 등)을 구현하고, 더 복잡한 데이터셋과 문제를 해결하는 도전을 이어갈 수 있을 것입니다.

**다음 학습 추천:**

*   **Convolutional Neural Network (CNN):** 이미지 처리에 특화된 강력한 네트워크입니다. PyTorch의 `nn.Conv2d`, `nn.MaxPool2d` 등을 사용하여 MNIST, CIFAR-10 등의 이미지 분류 성능을 크게 향상시킬 수 있습니다.
*   **PyTorch Flow 깊이 이해:** `autograd` (자동 미분), `Tensor` 연산, 커스텀 `Dataset` 및 `DataLoader` 생성 등을 더 깊이 학습하여 PyTorch를 자유자재로 활용하는 능력을 키웁니다.
*   **모델 저장 및 불러오기:** 학습된 모델을 파일로 저장하고 나중에 다시 불러와서 사용하거나 이어서 학습하는 방법을 배웁니다.
*   **하이퍼파라미터 튜닝 도구 사용:** Optuna, Ray Tune 등 하이퍼파라미터 최적화 도구를 사용하여 자동으로 최적의 학습 파라미터를 찾는 방법을 배웁니다.

